In [45]:
import autogen
from autogen import config_list_from_json

In [46]:
autogen.__version__

'0.2.2'

In [47]:
config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

## Construct Agents

In [48]:
llm_config = {
    'config_list': config_list,
    'timeout': 120,
    'temperature': 0,
    # 'cache_seed': 42
}

In [49]:
user_proxy = autogen.UserProxyAgent(
    name='CEO',
    system_message="CEO. Interact with Product Manager to discuss product requirements." \
                    "Plan execution needs to be approved by CEO.",
    code_execution_config=False,
)

product_manager = autogen.AssistantAgent(
    name="Product Manager",
    system_message='''
        Product Manager. 
        Listens to suggestions from CEO and come up with a Product Requirement Design (PRD).
        The PRD consists of User Stories for the features as well as the acceptance criteria.
        Revise the PRD based on feedback from CEO, UI/UX Designer and Engineer, until CEO approval.
        The PRD may involve an Engineer who can write code and a UI/UX Designer who doesn't write code.
        Explain the PRD first. 
        Be clear which step is performed by an Engineer, and which step is performed by a UI/UX Designer.
        After the PRD is approved, share it with the Data Analyst to come up with useful product metrics and how to measure success.
    ''',
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message='''
        Engineer. 
        You follow an approved plan. 
        You write python/shell code to solve tasks. 
        Wrap the code in a code block that specifies the script type. 
        The user can't modify your code. 
        So do not suggest incomplete code which requires others to modify. 
        Don't use a code block if it's not intended to be executed by the executor.
        Don't include multiple code blocks in one response. 
        Do not ask others to copy and paste the result. 
        Check the execution result returned by the executor.
        If the result indicates there is an error, fix the error and output the code again. 
        Suggest the full code instead of partial code or code changes. 
        If the error can't be fixed or if the task is not solved even after the code is executed successfully, 
        analyze the problem, revisit your assumption, collect additional info you need, 
        and think of a different approach to try.
''',
)

ui_ux_designer = autogen.AssistantAgent(
    name="UI/UX Designer",
    llm_config=llm_config,
    system_message='''
        UI/UX Designer. 
        You have good understanding of product requirements. 
        You emphatize with the users and are able to suggest good useful features.
        You are able to conduct research on product features and communicate it back to the Product Manager.
        You also criticize on features that may not add value or does not provide good user experience.
    ''',
)

executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Executor. Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "code"},
)

data_analyst = autogen.AssistantAgent(
    name="Data Analyst",
    llm_config=llm_config,
    system_message='''
        Data Analyst.
        You are a critical thinker and know what metrics to look for to measure a product feature success.
        You will write a report on the possible metrics, and how they can add value to the company in terms of revenue, user engagements etc.
    ''',
)

groupchat = autogen.GroupChat(agents=[user_proxy, engineer, product_manager, data_analyst, executor, ui_ux_designer], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [50]:
user_proxy.initiate_chat(
    manager,
    message="""
come up with a job portal to search for white collar jobs targetting the Thailand market. Generate the product requirements, then prepare a pitch deck presentation too.
""",
)

CEO (to chat_manager):


come up with a job portal to search for white collar jobs targetting the Thailand market. Generate the product requirements, then prepare a pitch deck presentation too.


--------------------------------------------------------------------------------
Product Manager (to chat_manager):

**Product Requirements**

* **Target Audience:** White-collar professionals in Thailand
* **Job Search:**
    * Users can search for jobs by keyword, location, industry, and other criteria.
    * Search results should be relevant and ranked by relevance.
    * Users can save and manage their job searches.
* **Job Listings:**
    * Employers can post job listings with detailed information, including job description, qualifications, and benefits.
    * Job listings should be easy to read and understand.
    * Employers can manage their job listings and track applicant activity.
* **Resumes and Applications:**
    * Job seekers can create and manage their resumes on the platform.
 